## Data Updation

#### The collected data (using the DataCollection code) needed to be updated after two weeks for more information regarding the other peoples reations to the tweet like the num. of likes or retweets

In [3]:
import json
import csv
import sys
import time
import pandas as pd
import ast
import numpy as np
import collections
import os
import math
import collections
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA,TruncatedSVD
import re
import random
from emoji_function import demojize
from myUtility import getTwitter, filePrepration, readData

In [18]:
def update(rootAddress, fileName, ids, twitter):
    
    """ 
        Update the data by retrieving the newest version of the tweet.
        
        Params:
          ids...........A list of integers, representing the unique id
              of each tweet in the dataset.
          fileName......A string representing the relationship
              category that been updated.
          twitter.......An object representing the Twitter API handler
          rootAddress...A string representing the rood directory 
    """
    
    filePrepration(rootAddress, [fileName])
    
    numTweets = len(ids)
    counter = 0
    tweets = []
    
    while (numTweets - counter) >= 100:
        """  
            update 100 tweets at a time using their id ('statuses/lookup' rate limitation = 
            100 id in each request)
        """
        update = twitter.request('statuses/lookup', {'id': ",".join(ids[counter:counter+100])})
        try:
            ## store the updated tweets in the file representing the relationship type
            for tweet in update.get_iterator():
                filePath = rootAddress + fileName + ".json"
                with open( filePath, mode='a', encoding='utf-8') as feedsjson:
                    feedsjson.write(json.dumps(tweet)+"\n")
                    feedsjson.close()
            counter += 100
            
        ## handle the twitter rate limitation ( sleep for 15 mins )
        except Exception as e: 
            print("---------------")
            print(e)
            print("---------------")
            print('Got error %s \nsleeping for 15 minutes.' % update.text)
            print(update.status_code)
            sys.stderr.flush()
            time.sleep(61 * 15)
            twitter = getTwitter('twitter.cfg')
    
    ## update the last batch of the tweets with size < 100
    try:
        update = twitter.request('statuses/lookup', {'id': ",".join(ids[counter:numTweets])})
        
    ## handle the twitter rate limitation ( sleep for 15 mins )
    except Exception as e: 
            print("---------------")
            print(e)
            print("---------------")
            print('Got error %s \nsleeping for 15 minutes.' % update.text)
            print(update.status_code)
            sys.stderr.flush()
            time.sleep(61 * 15)
            twitter = getTwitter('twitter.cfg')
            update = twitter.request('statuses/lookup', {'id': ",".join(ids[counter:numTweets])})

    for tweet in update.get_iterator(): 
        filePath = rootAddress + fileName + ".json"
        with open( filePath, mode='a', encoding='utf-8') as feedsjson:
                feedsjson.write(json.dumps(tweet)+"\n")
                feedsjson.close()


In [19]:
if __name__ == "__main__":
    
    rootAddress = "./"
    relationship_categories = [
        "dual_friendship", 
        "no_friendship",
        "target_follow_sender_friendship",
        "sender_follow_target_friendship"
    ]
    
    twitter = getTwitter('twitter.cfg')
    print('Established Twitter connection.')

    for relationship_type in relationship_categories:
        data = readData(rootAddress, relationship_type)
        relationship_pd = pd.DataFrame(data)
        relationship_tweet_ids = [str(tweet['id']) for index, tweet in relationship_pd.iterrows()] 
        update(rootAddress, relationship_type, relationship_tweet_ids, twitter)

Established Twitter connection.
